In [ ]:
from os import path, listdir
import gc

import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import datasets, layers, models, callbacks, optimizers
from sklearn.metrics import confusion_matrix

## Loading Dataset

In [ ]:
dataset_dir = path.join('stackedImagesGADFbw')

In [ ]:
def get_dataset(dataset_type, dataset_dir):
    labels = pd.read_csv(path.join(dataset_dir, 'y_' + dataset_type + '.txt'), header=None)
    dataset_X = list()
    dataset_y = list()
    for sample in listdir(path.join(dataset_dir, dataset_type)):
        dataset_X.append(np.load(path.join(dataset_dir, dataset_type, sample)))
        dataset_y.append(labels.iloc[int(sample.split('.')[0]) - 1][0])
    return np.array(dataset_X), to_categorical(np.array(dataset_y) - 1)

In [ ]:
X_train, y_train = get_dataset('train', dataset_dir)

In [ ]:
X_test, y_test = get_dataset('test', dataset_dir)

## Building Model

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.experimental.preprocessing.Rescaling(1./255, input_shape=(128, 128, 27)))
    
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 27)))
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPooling2D())
    
    # model.add(layers.Dropout(0.5))
    model.add(layers.Flatten())
    model.add(layers.Reshape((-1, 1)))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))

    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [ ]:
model = build_model()

In [ ]:
es = callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
model.fit(X_train, y_train, epochs=80, verbose=True, batch_size=64, validation_split=0.1, callbacks=[es])

In [ ]:
model.evaluate(X_test, y_test)

## Confusion Matrix

In [ ]:
predicted_labels = model.predict(X_test)
predicted_labels_normal = np.argmax(predicted_labels, axis=1)
correct_labels_normal = np.argmax(y_test, axis=1)

In [ ]:
confusion_matrix(correct_labels_normal, predicted_labels_normal)